In [1]:
import os
import pandas as pd
import traceback

from pathlib import Path
from datetime import datetime, timedelta
from itertools import product

In [2]:
base_dir = Path(r"\\192.168.10.221\Daten_flexibel_02\simbev_results\eDisGo_object_files_final")

scenarios = [
    "NEP_C_2035",
    "Reference_2050",
    "Electrification_2050",
    "Electrification_2050_sensitivity_low_work",
]

grid_ids = [
    "176",
    "177",
    "1056",
    "1690",
    "1811",
    "2534",
]

strategy = "dumb"

sub_dir = "timeseries"

data_dirs = [
    Path(os.path.join(base_dir, scenario, grid_id, strategy, sub_dir))
    for scenario in scenarios for grid_id in grid_ids
]

In [3]:
def get_days(
        grid_id,
        mode="weeks",
):
    try:
        s = pd.read_csv(
            os.path.join(
                r"\\192.168.10.221\Daten_flexibel_02\simbev_results",
                "extreme_weeks.csv",
            ),
            index_col=[0],
            parse_dates=[1,2,3,4],
        ).loc[int(grid_id)]

        if mode == "days":
            days = []

            for ts in [s.start_week_low, s.start_week_high]:
                for i in range(7):
                    days.append(ts + timedelta(days=i))

        elif mode == "weeks":
            days = [s.start_week_low, s.start_week_high]

        return days
    except:
        traceback.print_exc()
        
def cartesian_product(arr1, arr2):  
    # return the list of all the computed tuple 
    # using the product() method 
    return list(product(arr1, arr2))

def cartesian_product_3(arr1, arr2, arr3):  
    # return the list of all the computed tuple 
    # using the product() method 
    return list(product(arr1, arr2, arr3))

In [4]:
# load and generation
try:
    df_load_gen = pd.read_csv("load_and_generation_per_grid.csv", index_col=[0])

except:
    df_load_gen = pd.DataFrame(
        grid_ids,
        columns=["grid_id"]
    ).set_index("grid_id")

    df_load_gen = df_load_gen.assign(
        gen_fix_week_low=0,
        gen_fee_week_low=0,
        load_week_low=0,
        gen_fix_week_high=0,
        gen_fee_week_high=0,
        load_week_high=0,
    )

    gen_f = r"generators_active_power.csv"
    load_f = r"loads_active_power.csv"

    for data_dir in data_dirs[:6]:
        grid_id = data_dir.parts[-3]

        days = get_days(grid_id)

        df_gen = pd.read_csv(os.path.join(data_dir, gen_f), index_col=[0], parse_dates=True)

        df_load = pd.read_csv(os.path.join(data_dir, load_f), index_col=[0], parse_dates=True)

        for count, day in enumerate(days):
            df_gen_day = df_gen.loc[day:day + timedelta(days=7)]

            cols_fix = [col for col in df_gen_day.columns if ("hydro" in col or "bio" in col)]

            cols_fee = [col for col in df_gen_day.columns if ("solar" in col or "wind" in col)]

            if len(cols_fix) + len(cols_fee) != len(df_gen_day.columns):
                print("cols do not match in grid {}".format(grid_id))

            df_load_day = df_load.loc[day:day + timedelta(days=7)]

            if count == 0:
                df_load_gen.at[grid_id, "gen_fix_week_low"] = df_gen_day[cols_fix].sum().sum()
                df_load_gen.at[grid_id, "gen_fee_week_low"] = df_gen_day[cols_fee].sum().sum()
                df_load_gen.at[grid_id, "load_week_low"] = df_load_day.sum().sum()

            else:
                df_load_gen.at[grid_id, "gen_fix_week_high"] = df_gen_day[cols_fix].sum().sum()
                df_load_gen.at[grid_id, "gen_fee_week_high"] = df_gen_day[cols_fee].sum().sum()
                df_load_gen.at[grid_id, "load_week_high"] = df_load_day.sum().sum()


    df_load_gen = df_load_gen.divide(4).round(1)

    df_load_gen.to_csv("load_and_generation_per_grid.csv")


In [5]:
# charging points
try:
    df_cps = pd.read_csv("cp_demand_per_grid.csv", index_col=[0])
    
except:
    data = cartesian_product(scenarios, grid_ids)

    df_cps = pd.DataFrame(data=data, columns=["scenario", "grid_id"])
    df_cps["demand_week_low"] = 0
    df_cps["demand_week_high"] = 0

    cp_f = r"charging_points_active_power.csv"

    for data_dir in data_dirs:
        grid_id = data_dir.parts[-3]

        scenario = data_dir.parts[-4]

        days = get_days(grid_id)

        df_cp = pd.read_csv(os.path.join(data_dir, cp_f), index_col=[0], parse_dates=True)

        for count, day in enumerate(days):
            df_cp_day = df_cp.loc[day:day + timedelta(days=7)]

            if count == 0:
                df_cps.loc[
                    (df_cps.scenario == scenario) &
                    (df_cps.grid_id == grid_id)
                ] = df_cps.loc[
                    (df_cps.scenario == scenario) &
                    (df_cps.grid_id == grid_id)
                ].assign(
                    demand_week_low=df_cp_day.sum().sum()
                )

            else:
                df_cps.loc[
                    (df_cps.scenario == scenario) &
                    (df_cps.grid_id == grid_id)
                ] = df_cps.loc[
                    (df_cps.scenario == scenario) &
                    (df_cps.grid_id == grid_id)
                ].assign(
                    demand_week_high=df_cp_day.sum().sum()
                )

    df_cps.demand_week_low = df_cps.demand_week_low.divide(4).round(1)
    df_cps.demand_week_high = df_cps.demand_week_high.divide(4).round(1)

    df_cps.to_csv("cp_demand_per_grid.csv")


In [7]:
# curtailment
strategies = ["dumb", "grouped", "reduced", "residual"]

data = cartesian_product_3(scenarios, grid_ids, strategies)

df_curtailemt = pd.DataFrame(data=data, columns=["scenario", "grid_id", "strategy"])

df_curtailemt["curtailment_nv_gen_week_low"] = 0
df_curtailemt["curtailment_mv_gen_week_low"] = 0
df_curtailemt["curtailment_nv_load_week_low"] = 0
df_curtailemt["curtailment_mv_load_week_low"] = 0
df_curtailemt["curtailment_fix_gen_week_low"] = 0
df_curtailemt["curtailment_fee_gen_week_low"] = 0
df_curtailemt["curtailment_cp_load_week_low"] = 0
df_curtailemt["curtailment_other_load_week_low"] = 0

df_curtailemt["curtailment_nv_gen_week_high"] = 0
df_curtailemt["curtailment_mv_gen_week_high"] = 0
df_curtailemt["curtailment_nv_load_week_high"] = 0
df_curtailemt["curtailment_mv_load_week_high"] = 0
df_curtailemt["curtailment_fix_gen_week_high"] = 0
df_curtailemt["curtailment_fee_gen_week_high"] = 0
df_curtailemt["curtailment_cp_load_week_high"] = 0
df_curtailemt["curtailment_other_load_week_high"] = 0

sub_sub_dir = r"weekly_curtailment_v1"

cur_f = r"{}_{}_{}_curtailment.csv"

cur_gen_f = r"{}_{}_{}_curtailment_ts_per_gen.csv"

cur_load_f = r"{}_{}_{}_curtailment_ts_per_load.csv"

grid_ids_calculated = [
#     "176",
    "177",
    "1056",
    "1690",
#     "1811",
#     "2534",
]

for data_dir in data_dirs:    
    grid_id = data_dir.parts[-3]
    
    scenario = data_dir.parts[-4]
    
    if grid_id in grid_ids_calculated: # TODO
        days = get_days(grid_id)
        
        for strategy in strategies:
            strategy_dir = Path(
                os.path.join(
                    data_dir.parent.parent,
                    strategy,
                    sub_sub_dir
                )
            )
            
            for count, day in enumerate(days):
                day_string = day.strftime("%Y-%m-%d")
                
                df_cur = pd.read_csv(os.path.join(strategy_dir, cur_f.format(scenario, strategy, day_string)), index_col=[0])
                
                df_cur_gen = pd.read_csv(os.path.join(strategy_dir, cur_gen_f.format(scenario, strategy, day_string)), index_col=[0])
                
                df_cur_load = pd.read_csv(os.path.join(strategy_dir, cur_load_f.format(scenario, strategy, day_string)), index_col=[0])
                
                curtailment_nv_gen = df_cur.at["lv_problems", "feed-in"] + df_cur.at["lv_convergence_problems", "feed-in"]
                curtailment_mv_gen = df_cur.at["mv_problems", "feed-in"] + df_cur.at["mv_convergence_problems", "feed-in"]
                curtailment_nv_load = df_cur.at["lv_problems", "load"] + df_cur.at["lv_convergence_problems", "load"]
                curtailment_mv_load = df_cur.at["mv_problems", "load"] + df_cur.at["mv_convergence_problems", "load"]
                
                cols_fix = [col for col in df_cur_gen.columns if ("hydro" in col or "bio" in col)]

                cols_fee = [col for col in df_cur_gen.columns if ("solar" in col or "wind" in col)]
                
                if len(cols_fix) + len(cols_fee) != len(df_cur_gen.columns):
                    print("cols do not match in grid {} for gen".format(grid_id))
                
                curtailment_fix_gen = df_cur_gen[cols_fix].sum().sum()
                curtailment_fee_gen = df_cur_gen[cols_fee].sum().sum()
                
                cols_cp = [col for col in df_cur_load.columns if "charging" in col.lower()]

                cols_other = [col for col in df_cur_load.columns if "charging" not in col.lower()]
                
                if len(cols_cp) + len(cols_other) != len(df_cur_load.columns):
                    print("cols do not match in grid {} for load".format(grid_id))
                    
                curtailment_cp_load = df_cur_load[cols_cp].sum().sum()
                curtailment_other_load = df_cur_load[cols_other].sum().sum()
                
                if count == 0:
                    df_curtailemt.loc[
                        (df_curtailemt.scenario == scenario) &
                        (df_curtailemt.grid_id == grid_id) &
                        (df_curtailemt.strategy == strategy)
                    ] = df_curtailemt.loc[
                        (df_curtailemt.scenario == scenario) &
                        (df_curtailemt.grid_id == grid_id) &
                        (df_curtailemt.strategy == strategy)
                    ].assign(
                        curtailment_nv_gen_week_low=curtailment_nv_gen,
                        curtailment_mv_gen_week_low=curtailment_mv_gen,
                        curtailment_nv_load_week_low=curtailment_nv_load,
                        curtailment_mv_load_week_low=curtailment_mv_load,
                        curtailment_fix_gen_week_low=curtailment_fix_gen,
                        curtailment_fee_gen_week_low=curtailment_fee_gen,
                        curtailment_cp_load_week_low=curtailment_cp_load,
                        curtailment_other_load_week_low=curtailment_other_load,
                    )
                
                else:
                    df_curtailemt.loc[
                        (df_curtailemt.scenario == scenario) &
                        (df_curtailemt.grid_id == grid_id) &
                        (df_curtailemt.strategy == strategy)
                    ] = df_curtailemt.loc[
                        (df_curtailemt.scenario == scenario) &
                        (df_curtailemt.grid_id == grid_id) &
                        (df_curtailemt.strategy == strategy)
                    ].assign(
                        curtailment_nv_gen_week_high=curtailment_nv_gen,
                        curtailment_mv_gen_week_high=curtailment_mv_gen,
                        curtailment_nv_load_week_high=curtailment_nv_load,
                        curtailment_mv_load_week_high=curtailment_mv_load,
                        curtailment_fix_gen_week_high=curtailment_fix_gen,
                        curtailment_fee_gen_week_high=curtailment_fee_gen,
                        curtailment_cp_load_week_high=curtailment_cp_load,
                        curtailment_other_load_week_high=curtailment_other_load,
                    )
                
df_curtailemt.iloc[:, 3:] = df_curtailemt.iloc[:, 3:].divide(4).round(1)
df_curtailemt.to_csv("curtailment_per_type_and_voltage_level.csv")

In [54]:
cols = [
    "scenario",
    "strategy",
    "curtailment_nv_load_week_high",
    "curtailment_mv_load_week_high",
    "curtailment_nv_gen_week_high",
    "curtailment_mv_gen_week_high",
    "curtailment_nv_load_week_low",
    "curtailment_mv_load_week_low",
    "curtailment_nv_gen_week_low",
    "curtailment_mv_gen_week_low",
]

for grid_id in grid_ids_calculated:

    df = df_curtailemt.loc[df_curtailemt.grid_id == grid_id][cols]

    df_profile = pd.DataFrame(
        columns=df.scenario.unique().tolist(),
        index=range(32)
    )

    scenario_list = []

    for scenario in df.scenario.unique():
        for strategy in df.strategy.unique():
            row = df.loc[(df.scenario == scenario) & (df.strategy == strategy)].iloc[0, 2:6].tolist()

            scenario_list.extend(row)

        for strategy in df.strategy.unique():
            row = df.loc[(df.scenario == scenario) & (df.strategy == strategy)].iloc[0, 6:].tolist()

            scenario_list.extend(row)

        df_profile[scenario] = scenario_list.copy()

        scenario_list = []
    
    df_profile.to_csv("{}_profile_long.csv".format(grid_id))
